In [1]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import re
import random
import json

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import numpy as np
from torch.utils.data import TensorDataset, DataLoader, RandomSampler

import warnings
warnings.filterwarnings("ignore") 

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
from transformers import GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("openai-community/gpt2")
SOS_token = tokenizer.bos_token_id  # 使用 GPT2 的 <bos> 作为 SOS
EOS_token = tokenizer.eos_token_id  # 使用 GPT2 的 <eos> 作为 EOS
vocab_size = len(tokenizer)

# For English, lowercase, trim, and remove non-letter characters
def normalizeEng(s):
    '''
    process the English sentence
    '''
    s = s.lower().strip() # remove leading and trailing spaces then tokenize
    s = re.sub(r"([.!?])", r" \1", s) # add spaces around .!? to separate them from words
    s = re.sub(r"[^a-zA-Z!?]+", r" ", s) # replace other characters with spaces
    return s.strip()
# For Chinese, trim, and remove other characters
def normalizeChi(s):
    s = s.strip()
    s = re.sub(r"([.!?。])", r" \1 ", s)
    s = re.sub(r"[^\u4e00-\u9fff\sa-zA-Z!！?。.]+", r" ", s)
    return s.strip()

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [3]:
def readLangs():
    '''
    read txt file, process sentence, convert them into pairs
    '''
    print("Reading lines...")

    # Read the file and split into lines
    lines = open(dir, encoding='utf-8').read().strip().split('\n')

    # Split every line into chinese english pairs and normalize
    pairs = [[normalizeChi(l.split('\t')[1]),normalizeEng(l.split('\t')[0])] for l in lines]

    return pairs

In [4]:
def indexesFromSentence(sentence):
    return tokenizer(sentence)["input_ids"]

def tensorFromSentence(sentence):
    indexes = indexesFromSentence(sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(1, -1)

def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(pair[0])
    target_tensor = tensorFromSentence(pair[1])
    return (input_tensor, target_tensor)

from torch.utils.data import random_split
torch.manual_seed(42)
def get_dataloader(batch_size, MAX_LENGTH=100):
    pairs = readLangs()

    n = len(pairs)
    input_ids = np.zeros((n, MAX_LENGTH), dtype=np.int32)
    target_ids = np.zeros((n, MAX_LENGTH), dtype=np.int32)

    for idx, (inp, tgt) in enumerate(pairs):
        inp_ids = indexesFromSentence(inp)
        tgt_ids = indexesFromSentence(tgt)
        inp_ids.append(EOS_token)
        tgt_ids.append(EOS_token)
        input_ids[idx, :len(inp_ids)] = inp_ids
        target_ids[idx, :len(tgt_ids)] = tgt_ids

    data = TensorDataset(torch.LongTensor(input_ids).to(device),
                               torch.LongTensor(target_ids).to(device))

    train_size = int(0.9 * len(data))
    test_size = len(data) - train_size

    train_dataset, test_dataset = random_split(data, [train_size, test_size])

    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return train_dataloader, test_dataloader

In [5]:
import time

def train_epoch(dataloader, encoder, decoder, encoder_optimizer,
                decoder_optimizer, criterion):
    total_loss = 0
    start_time = time.time()
    s_time = time.time()

    for i, data in enumerate(dataloader):
        input_tensor, target_tensor = data

        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()

        encoder_outputs, encoder_hidden = encoder(input_tensor)
        decoder_outputs, _, _ = decoder(encoder_outputs, encoder_hidden, target_tensor)

        loss = criterion(
            decoder_outputs.view(-1, decoder_outputs.size(-1)),
            target_tensor.view(-1)
        )
        loss.backward()

        encoder_optimizer.step()
        decoder_optimizer.step()

        total_loss += loss.item()

        if i % 100 == 0:
            print(f'Within one epoch: training {i}/{len(dataloader)} | Time: {time.time() - s_time:.2f}s')
            s_time = time.time()

    end_time = time.time()
    epoch_time = end_time - start_time
    print(f'\nOne Epoch finished | Total time: {epoch_time:.2f}s | Average loss: {total_loss / len(dataloader):.4f}')

    return total_loss / len(dataloader)

## Encoder

In [6]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, dropout_p=0.1):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, input):
        embedded = self.dropout(self.embedding(input))
        output, hidden = self.gru(embedded)
        return output, hidden

## Decoder with attention

In [7]:
class BahdanauAttention(nn.Module):
    def __init__(self, hidden_size):
        super(BahdanauAttention, self).__init__()
        self.Wa = nn.Linear(hidden_size, hidden_size)
        self.Ua = nn.Linear(hidden_size, hidden_size)
        self.Va = nn.Linear(hidden_size, 1)

    def forward(self, query, keys):
        scores = self.Va(torch.tanh(self.Wa(query) + self.Ua(keys)))
        scores = scores.squeeze(2).unsqueeze(1)

        weights = F.softmax(scores, dim=-1)
        context = torch.bmm(weights, keys)

        return context, weights

class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1):
        super(AttnDecoderRNN, self).__init__()
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.attention = BahdanauAttention(hidden_size)
        self.gru = nn.GRU(2 * hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, encoder_outputs, encoder_hidden, target_tensor=None):
        batch_size = encoder_outputs.size(0)
        decoder_input = torch.empty(batch_size, 1, dtype=torch.long, device=device).fill_(SOS_token)
        decoder_hidden = encoder_hidden
        decoder_outputs = []
        attentions = []

        for i in range(MAX_LENGTH):
            decoder_output, decoder_hidden, attn_weights = self.forward_step(
                decoder_input, decoder_hidden, encoder_outputs
            )
            decoder_outputs.append(decoder_output)
            attentions.append(attn_weights)

            if target_tensor is not None:
                # Teacher forcing: Feed the target as the next input
                decoder_input = target_tensor[:, i].unsqueeze(1) # Teacher forcing
            else:
                # Without teacher forcing: use its own predictions as the next input
                _, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze(-1).detach()  # detach from history as input

        decoder_outputs = torch.cat(decoder_outputs, dim=1)
        decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)
        attentions = torch.cat(attentions, dim=1)

        return decoder_outputs, decoder_hidden, attentions


    def forward_step(self, input, hidden, encoder_outputs):
        embedded =  self.dropout(self.embedding(input))

        query = hidden.permute(1, 0, 2)
        context, attn_weights = self.attention(query, encoder_outputs)
        input_gru = torch.cat((embedded, context), dim=2)

        output, hidden = self.gru(input_gru, hidden)
        output = self.out(output)

        return output, hidden, attn_weights

## Training

In [8]:
import time
import math

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [9]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np

def showPlot(plot_losses, save_plot_path="training_loss.png"):
    plt.figure()
    fig, ax = plt.subplots()
    # 设置y轴的刻度间隔
    loc = ticker.MultipleLocator(base=0.01)
    ax.yaxis.set_major_locator(loc)
    # 设置y轴的刻度格式，保留两位小数
    formatter = ticker.FormatStrFormatter('%.2f')
    ax.yaxis.set_major_formatter(formatter)
    plt.plot(plot_losses.keys(), plot_losses.values())
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.title("Training Loss")
    plt.savefig(save_plot_path)  # 保存图像
    plt.show()

In [10]:
from tqdm import tqdm
import pandas as pd
import pickle

def train(train_dataloader, encoder, decoder, n_epochs, learning_rate=0.001,save_pkl_path="losses.pkl"):
    start = time.time()
    losses = {}

    encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)
    criterion = nn.NLLLoss()

    for epoch in tqdm(range(1, n_epochs + 1),desc="Training Epochs..."):
        loss = train_epoch(train_dataloader, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
        losses[epoch] = loss
        print('%s (%d %d%%) loss: %.4f' % (timeSince(start, epoch / n_epochs),
                                        epoch, epoch / n_epochs * 100, loss))
        print(f'==== Finished training of {epoch}/{n_epochs} epochs，loss is {loss} ====')

    showPlot(losses)
    with open(save_pkl_path, 'wb') as f:
        pickle.dump(losses, f)

## Evaluation & attention visualization

In [11]:
def tensor2sentence(input_tensor):
    return tokenizer.decode(input_tensor)
def evaluate(encoder, decoder, input_tensor):
    with torch.no_grad():
        encoder_outputs, encoder_hidden = encoder(input_tensor)
        decoder_outputs, decoder_hidden, decoder_attn = decoder(encoder_outputs, encoder_hidden)

        _, topi = decoder_outputs.topk(1)
        decoded_ids = topi.squeeze()[0]

        decoded_sentence = tensor2sentence(decoded_ids)
    return decoded_sentence, decoder_attn

In [44]:
# visualize attention
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
myfont = fm.FontProperties(fname="/kaggle/input/simhei/SIMHEI.TTF") #Kaggle需要使用上传的黑体

def showAttention(input_sentence, output_words, attentions):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(attentions.cpu().numpy(), cmap='plasma')
    fig.colorbar(cax)

    # Set up axes
    ax.set_xticklabels([''] + list(input_sentence) +
                       ['<EOS>'], rotation=90,fontproperties=myfont)
    ax.set_yticklabels([''] + output_words.split(' ')+
                       ['<EOS>'])

    # Show label at every tick
    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.savefig(f'attention_map_{input_sentence}.png')
    plt.show()

In [52]:
import json
def evaluateRandomly(dataloader, encoder, decoder, save_path='evaluation.json'):
    results = []  # 创建一个空列表来存储结果
    for data in dataloader:
        input_tensor, target_tensor = data
        input_sentence = tensor2sentence(input_tensor[0]).split('<|endoftext|>', 1)[0]
        target_sentence = tensor2sentence(target_tensor[0]).split('<|endoftext|', 1)[0]
        output_sentence, decoder_attn = evaluate(encoder, decoder, input_tensor)
        output_sentence = output_sentence.split('<|endoftext|', 1)[0]

        # 打印结果
        print('>', input_sentence)
        print('=', target_sentence)
        print('<', output_sentence)
        print('')
        
        showAttention(input_sentence, output_sentence, decoder_attn[0, :len(output_sentence.split(' '))+1, :len(list(input_sentence))+1])
        
        # 将结果添加到列表中
        results.append({
            'input': input_sentence,
            'target': target_sentence,
            'output': output_sentence.split('<|endoftext|>', 1)[0]
        })

    # 将结果列表写入JSON文件
    with open(save_path, 'w', encoding='utf-8') as f:
        json.dump(results, f, ensure_ascii=False, indent=4)

## main

In [ ]:
hidden_size = 128
batch_size = 32
dir = '/kaggle/input/cmn-eng/cmn.txt'
MAX_LENGTH = 100

train_dataloader, test_dataloader = get_dataloader(batch_size)

encoder = EncoderRNN(vocab_size, hidden_size).to(device)
decoder = AttnDecoderRNN(hidden_size, vocab_size).to(device)

train(train_dataloader, encoder, decoder, 80)

In [ ]:
# 保存模型
encoder_path = 'encoder_model.pth'
decoder_path = 'decoder_model.pth'

torch.save(encoder.state_dict(), encoder_path)
torch.save(decoder.state_dict(), decoder_path)

print(f"Encoder model saved to {encoder_path}")
print(f"Decoder model saved to {decoder_path}")

In [ ]:
# evaluation
encoder.eval()
decoder.eval()
evaluateRandomly(test_dataloader,encoder, decoder)

# Load and eval

In [14]:
# 创建模型实例
hidden_size = 128
batch_size = 32
dir = '/kaggle/input/cmn-eng/cmn.txt'
MAX_LENGTH = 100

train_dataloader, test_dataloader = get_dataloader(batch_size)

encoder = EncoderRNN(vocab_size, hidden_size).to(device)
decoder = AttnDecoderRNN(hidden_size, vocab_size).to(device)

# 加载模型参数
encoder_path = '/kaggle/input/rnn-attention-80/encoder_model.pth'
decoder_path = '/kaggle/input/rnn-attention-80/decoder_model.pth'

# 加载参数到模型
encoder.load_state_dict(torch.load(encoder_path))
decoder.load_state_dict(torch.load(decoder_path))

# 将模型设置为评估模式
encoder.eval()
decoder.eval()


Reading lines...


AttnDecoderRNN(
  (embedding): Embedding(50257, 128)
  (attention): BahdanauAttention(
    (Wa): Linear(in_features=128, out_features=128, bias=True)
    (Ua): Linear(in_features=128, out_features=128, bias=True)
    (Va): Linear(in_features=128, out_features=1, bias=True)
  )
  (gru): GRU(256, 128, batch_first=True)
  (out): Linear(in_features=128, out_features=50257, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
)

In [53]:
evaluateRandomly(test_dataloader,encoder, decoder)

> 你不該那樣跟湯姆講話 。
= you shouldn t talk to tom that way
< you shouldn t talk to tom that way

> 书桌上有一把剪刀 。
= there is a pair of scissors on the desk
< there is a pair of scissors on the desk

> 她企圖自殺 。
= she attempted suicide
< she tried to kill herself

> 我什么都不喜欢 。
= i don t like anything
< i don t like anything

> 您想吃点什么吗
= would you like to eat something ?
< would you like something to eat ?

> 把這本書放在架子的底部 。
= put the book on the bottom shelf
< put the book on the bottom shelf

> 他比我高两寸 。
= he is two inches taller than i am
< he is my younger than me

> 他是世界上最有錢的人 。
= he is the richest man on earth
< he is the richest man on earth

> 她叫什麼名字
= what s her name ?
< what s her name ?

> 你可能該動一動 。
= maybe you should move
< you may have been attention

> 我約會遲到了 。
= i was late for the appointment
< i will be late

> 年轻人应该尊敬老人 。
= the young should respect the old
< the young should respect the old

> 煤和天然氣是天然燃料 。
= coal and natural gas are natural fuels
< coal and natural gas are natural fuels
